In [ ]:
%pip install rdkit-pypi

In [18]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as pyplot
import seaborn as sns
%matplotlib inline

from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors

In [24]:
pd.set_option('display.max_columns', None)

train_data = pd.read_csv('train_II.csv')
test_data = pd.read_csv('test_II.csv')

In [7]:
train_data.sample(10)

,Id,Expected
73022,CCCCOC(=O)[C@@H](C)OC1=CC=C(C=C1)OC2=NC=C(C=C2...,2
8219,CC(C)C1=C(C=C(C=C1)C(=O)N=C(N)N)S(=O)(=O)C.CS(...,2
5981,C1=CC(=CC=C1N)Cl;1508,1
8927,CC(C)OP(=S)(OC(C)C)SCCNS(=O)(=O)C1=CC=CC=C1;1637,2
20482,C1(=C(C(C(=C1Cl)Cl)(Cl)Cl)Cl)Cl;1387,2
24744,CCN(CC1=CN=CC=C1)C(=O)C2=C(C(=CC=C2)Cl)[C@]3(C...,1
3533,CC(C)(C)C1=NN=C(S1)N(C)C(=O)NC;1507,2
25768,CCNC(=O)NC(=O)/C(=N/OC)/C#N;19,2
36515,C1=CC2=CC3=C(C(=CC=C3)O)C(=C2C(=C1)O)O;33,2
62011,CCC(=C1C(=O)CC(CC1=O)C(=O)[O-])[O-].[Ca+2];1641,2


In [8]:
test_data = test_data.rename(columns = {"x": "Id"})
test_data.sample(10)

,Id
8857,CC(C)(C)C1=CC=C(C=C1)O;2446
4595,C1=CC=C(C(=C1)O)O;1507
10795,CN1CN(C(=S)SC1)C;1383
7641,CO;1645
2820,CC(C)(C)C1CCC(CC1)O;1850
9253,C1=CC(=C(C=C1O)O)C(=O)C2=C(C=C(C=C2)O)O;1386
7502,CCC(C)N1C(=O)C(=C(NC1=O)C)Br;16
9857,CCOP(=S)(OCC)OC(C(Cl)(Cl)Cl)Cl;1622
4627,CCN(CC)N=O;1680
10171,C1=CC=C(C=C1)NC(=O)NC2=CC(=NC=C2)Cl;1509


In [10]:
train_data[["Chemical_Id", "Assay_Id"]] = train_data.Id.str.split(";", expand = True)
test_data[["Chemical_Id", "Assay_Id"]] = test_data.Id.str.split(";", expand = True)

train_data.sample(10)

,Id,Expected,Chemical_Id,Assay_Id
44047,CC(C)(C1=CC(=C(C(=C1)Br)O)Br)C2=CC(=C(C(=C2)Br...,2,CC(C)(C1=CC(=C(C(=C1)Br)O)Br)C2=CC(=C(C(=C2)Br...,28
13190,CN1CCN([C@H](C1)C2=CC=C(C=C2)F)CC3=CC(=CO3)C4=...,1,CN1CCN([C@H](C1)C2=CC=C(C=C2)F)CC3=CC(=CO3)C4=...,1376
73736,CCCCCCCCCCCC(=O)OCC(C)OC(=O)CCCCC(=O)O;27,2,CCCCCCCCCCCC(=O)OCC(C)OC(=O)CCCCC(=O)O,27
4654,CCCN(CCC)C(=O)SCC;1375,2,CCCN(CCC)C(=O)SCC,1375
74298,CC1=CC(=O)NC(=S)N1;40,2,CC1=CC(=O)NC(=S)N1,40
42031,CCCSP(=S)(OCC)OC1=CC=C(C=C1)SC;2,2,CCCSP(=S)(OCC)OC1=CC=C(C=C1)SC,2
62948,CN(C)C1=CC2=C(C=C1)N=C3C=CC(=[N+](C)C)C=C3S2.O...,2,CN(C)C1=CC2=C(C=C1)N=C3C=CC(=[N+](C)C)C=C3S2.O...,2453
62276,CNCC1=CC=C(C=C1)NC(=O)C2=CNC3=C2C(=O)CCC3;1852,2,CNCC1=CC=C(C=C1)NC(=O)C2=CNC3=C2C(=O)CCC3,1852
73609,CCCCC(CN1C=NC=N1)(C2=C(C=C(C=C2)Cl)Cl)O;1618,2,CCCCC(CN1C=NC=N1)(C2=C(C=C(C=C2)Cl)Cl)O,1618
43272,CCNC(=O)NC(=O)/C(=N/OC)/C#N;1508,1,CCNC(=O)NC(=O)/C(=N/OC)/C#N,1508


In [11]:
# filtering out 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]'
train_data= train_data[train_data.Chemical_Id != 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]']
test_data= test_data[test_data.Chemical_Id != 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]']

In [13]:
test_data.sample(10)

,Id,Chemical_Id,Assay_Id
5293,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,1621
2483,CC(C)CCCCCCOC(=O)CCCCC(=O)OCCCCCCC(C)C;1381,CC(C)CCCCCCOC(=O)CCCCC(=O)OCCCCCCC(C)C,1381
9190,C[C@]12CCC(=O)C=C1CC[C@@H]3[C@@H]2CC[C@]4([C@H...,C[C@]12CCC(=O)C=C1CC[C@@H]3[C@@H]2CC[C@]4([C@H...,28
7575,C1(=C(C(=C(C(=C1Cl)Cl)Cl)Cl)Cl)O;1510,C1(=C(C(=C(C(=C1Cl)Cl)Cl)Cl)Cl)O,1510
1193,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C#C)O)CCC4...,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@]2(C#C)O)CCC4...,1690
4297,CCCOC1=NN(C(=O)N1C)C(=O)[N-]S(=O)(=O)C2=CC=CC=...,CCCOC1=NN(C(=O)N1C)C(=O)[N-]S(=O)(=O)C2=CC=CC=...,1507
9606,COC1=CC(=NC(=N1)NC(=O)[N-]S(=O)(=O)C2=C(C=CC=N...,COC1=CC(=NC(=N1)NC(=O)[N-]S(=O)(=O)C2=C(C=CC=N...,27
8,C1=CC(=C(C=C1Cl)Cl)O;1388,C1=CC(=C(C=C1Cl)Cl)O,1388
5266,C1=CC=C(C=C1)N=NC2=CC=C(C=C2)N;2453,C1=CC=C(C=C1)N=NC2=CC=C(C=C2)N,2453
9544,CC(COCCCOC)OCCCOC(=O)C1=CC=CC=C1;40,CC(COCCCOC)OCCCOC(=O)C1=CC=CC=C1,40


In [20]:

# Define the "RDkit_descriptors" function that takes in a list of SMILES strings
def RDkit_descriptors(smiles):
    
    # Convert the SMILES strings into RDKit molecules and store them in a list called "mols"
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    
    # Create a molecular descriptor calculator that will be used to calculate the descriptors for each molecule
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    
    # Get the names of the descriptors that will be calculated
    desc_names = calc.GetDescriptorNames()
    
    # Initialize an empty list to store the calculated descriptor values for each molecule
    Mol_descriptors =[]
    
    # Iterate over each molecule in the "mols" list
    for mol in mols:
        # Add hydrogen atoms to the molecule
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule using the calculator object
        descriptors = calc.CalcDescriptors(mol)
        # Append the descriptor values to the "Mol_descriptors" list
        Mol_descriptors.append(descriptors)
    
    return Mol_descriptors,desc_names

# Calculate the descriptors for the training set
train_Mol_descriptors,desc_names = RDkit_descriptors(train_data.Chemical_Id)


In [22]:
# create a dataframe from the train descriptors
train_desc = pd.DataFrame(train_Mol_descriptors,columns=desc_names)
train_desc

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,8.874650,-2.987783,8.874650,0.765809,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,0,0,0,0,0
1,11.875100,-4.547647,11.875100,2.480493,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,0,0,0,0,0,0,0
2,8.736945,-6.030543,8.736945,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,0,0,0,0,0,0,0
3,10.885281,-3.359276,10.885281,0.170399,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,12.226619,-5.339907,12.226619,0.265375,0.712426,230.245,220.165,230.036128,82,0,...,0,1,0,0,0,0,0,0,0,0
75373,8.048957,-5.193498,8.048957,0.110750,0.720533,313.747,296.611,313.041677,104,0,...,0,0,0,0,0,0,0,0,0,0
75374,7.612361,-0.332130,7.612361,0.131389,0.596343,167.258,162.218,166.986341,50,0,...,0,0,0,0,0,1,0,0,0,0
75375,11.548424,-4.352546,11.548424,2.482554,0.519485,128.215,112.087,128.120115,54,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
test_Mol_descriptors,desc_names = RDkit_descriptors(test_data.Chemical_Id)

# create a dataframe from the test descriptors
test_desc = pd.DataFrame(test_Mol_descriptors,columns=desc_names)
test_desc

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,8.018449,-3.840318,8.018449,1.098125,0.624614,164.248,148.120,164.120115,66,0,...,0,0,0,0,0,0,0,0,0,0
1,13.253248,-6.190954,13.253248,0.749312,0.617511,431.452,414.316,431.056940,152,0,...,0,1,1,0,0,0,0,0,0,1
2,16.485402,-6.427849,16.485402,0.554422,0.224134,696.264,655.944,695.250845,254,0,...,0,1,0,0,0,0,0,0,0,0
3,10.745579,-4.559583,10.745579,0.000000,0.393203,201.244,197.212,200.949810,56,0,...,0,0,0,0,0,0,0,0,0,0
4,14.743838,-5.881101,14.743838,2.392476,0.639062,418.574,380.270,418.271924,168,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,13.677486,-5.541970,13.677486,1.263580,0.391778,408.376,389.224,408.129692,154,0,...,0,0,0,0,0,0,0,0,0,0
10990,12.722294,-3.388339,12.722294,0.917255,0.635519,320.344,304.216,320.104859,120,0,...,0,0,0,0,0,0,0,0,0,0
10991,13.186679,-5.902467,13.186679,0.638923,0.705120,381.370,366.250,381.074304,138,0,...,0,1,0,0,0,0,0,0,0,1
10992,11.701188,-5.111667,11.701188,0.820825,0.540194,220.976,213.920,219.945901,66,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
train_desc.replace([np.inf,-np.inf], np.nan,inplace=True)
test_desc.replace([np.inf,-np.inf], np.nan,inplace=True)

In [26]:
train_desc.fillna(train_desc.mean(), inplace=True)
test_desc.fillna(test_desc.mean(), inplace=True)

In [27]:
# Define a function called "morgan_fpts" that takes in a list of SMILES strings ("data")
def morgan_fpts(data):
    # Create an empty list called "Morgan_fpts" to store the Morgan fingerprints for each molecule
    Morgan_fpts = []
    
    # Loop over each SMILES string in the input "data" list
    for i in data:
        # Convert the SMILES string into an RDKit molecule object
        mol = Chem.MolFromSmiles(i) 
        # Generate a Morgan fingerprint for the molecule using a radius of 2 and a length of 512 bits
        fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,2,512)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)  
    
    # Convert the "Morgan_fpts" list into a NumPy array and return it
    return np.array(Morgan_fpts)

In [31]:
train_data[["Chemical_Id", "Assay_Id"]] = train_data.Id.str.split(";", expand = True)
train_data

,Id,Expected,Chemical_Id,Assay_Id
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644
1,CCCCCCCCC(=O)C;2451,2,CCCCCCCCC(=O)C,2451
2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-];1384,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384
3,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl;16,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16
4,[Na+].[I-];1856,2,[Na+].[I-],1856
...,...,...,...,...
75378,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N;33,2,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N,33
75379,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C;1632,1,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C,1632
75380,C1=CC=C2C(=C1)NC(=S)S2;1373,1,C1=CC=C2C(=C1)NC(=S)S2,1373
75381,CCCCC(CC)C=O;2,2,CCCCC(CC)C=O,2


In [34]:
train_data= train_data[train_data.Chemical_Id != 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]']

In [36]:
# Calculate the Morgan fingerprints for the training set
train_Morgan_fpts = morgan_fpts(train_data.Chemical_Id)
print(train_Morgan_fpts.shape)

# create a dataframe from the train fingerprints
train_fp = pd.DataFrame(train_Morgan_fpts,columns=['Col_{}'.format(i) for i in range(train_Morgan_fpts.shape[1])])
train_fp

(75377, 512)


,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Col_176,Col_177,Col_178,Col_179,Col_180,Col_181,Col_182,Col_183,Col_184,Col_185,Col_186,Col_187,Col_188,Col_189,Col_190,Col_191,Col_192,Col_193,Col_194,Col_195,Col_196,Col_197,Col_198,Col_199,Col_200,Col_201,Col_202,Col_203,Col_204,Col_205,Col_206,Col_207,Col_208,Col_209,Col_210,Col_211,Col_212,Col_213,Col_214,Col_215,Col_216,Col_217,Col_218,Col_219,Col_220,Col_221,Col_222,Col_223,Col_224,Col_225,Col_226,Col_227,Col_228,Col_229,Col_230,Col_231,Col_232,Col_233,Col_234,Col_235,Col_236,Col_237,Col_238,Col_239,Col_240,Col_241,Col_242,Col_243,Col_244,Col_245,Col_246,Col_247,Col_248,Col_249,Col_250,Col_251,Col_252,Col_253,Col_254,Col_255,Col_256,Col_257,Col_258,Col_259,Col_260,Col_261,Col_262,Col_263,Col_264,Col_265,Col_266,Col_267,Col_268,Col_269,Col_270,Col_271,Col_272,Col_273,Col_274,Col_275,Col_276,Col_277,Col_278,Col_279,Col_280,Col_281,Col_282,Col_283,Col_284,Col_285,Col_286,Col_287,Col_288,Col_289,Col_290,Col_291,Col_292,Col_293,Col_294,Col_295,Col_296,Col_297,Col_298,Col_299,Col_300,Col_301,Col_302,Col_303,Col_304,Col_305,Col_306,Col_307,Col_308,Col_309,Col_310,Col_311,Col_312,Col_313,Col_314,Col_315,Col_316,Col_317,Col_318,Col_319,Col_320,Col_321,Col_322,Col_323,Col_324,Col_325,Col_326,Col_327,Col_328,Col_329,Col_330,Col_331,Col_332,Col_333,Col_334,Col_335,Col_336,Col_337,Col_338,Col_339,Col_340,Col_341,Col_342,Col_343,Col_344,Col_345,Col_346,Col_347,Col_348,Col_349,Col_350,Col_351,Col_352,Col_353,Col_354,Col_355,Col_356,Col_357,Col_358,Col_359,Col_360,Col_361,Col_362,Col_363,Col_364,Col_365,Col_366,Col_367,Col_368,Col_369,Col_370,Col_371,Col_372,Col_373,Col_374,Col_375,Col_376,Col_377,Col_378,Col_379,Col_380,Col_381,Col_382,Col_383,Col_384,Col_385,Col_386,Col_387,Col_388,Col_389,Col_390,Col_391,Col_392,Col_393,Col_394,Col_395,Col_396,Col_397,Col_398,Col_399,Col_400,Col_401,Col_402,Col_403,Col_404,Col_405,Col_406,Col_407,Col_408,Col_409,Col_410,Col_411,Col_412,Col_413,Col_414,Col_415,Col_416,Col_417,Col_418,Col_419,Col_420,Col_421,Col_422,Col_423,Col_424,Col_425,Col_426,Col_427,Col_428,Col_429,Col_430,Col_431,Col_432,Col_433,Col_434,Col_435,Col_436,Col_437,Col_438,Col_439,Col_440,Col_441,Col_442,Col_443,Col_444,Col_445,Col_446,Col_447,Col_448,Col_449,Col_450,Col_451,Col_452,Col_453,Col_454,Col_455,Col_456,Col_457,Col_458,Col_459,Col_460,Col_461,Col_462,Col_463,Col_464,Col_465,Col_466,Col_467,Col_468,Col_469,Col_470,Col_471,Col_472,Col_473,Col_474,Col_475,Col_476,Col_477,Col_478,Col_479,Col_480,Col_481,Col_482,Col_483,Col_484,Col_485,Col_486,Col_487,Col_488,Col_489,Col_490,Col_491,Col_492,Col_493,Col_494,Col_495,Col_496,Col_497,Col_498,Col_499,Col_500,Col_501,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,0,1,0,0,0,0

In [39]:
test_data = test_data.rename(columns = {"x": "Id"})
test_data[["Chemical_Id", "Assay_Id"]] = test_data.Id.str.split(";", expand = True)
test_data= test_data[test_data.Chemical_Id != 'F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]']
test_data

,Id,Chemical_Id,Assay_Id
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,CC1=CC(=C(C=C1)C(C)(C)C)O,1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30
...,...,...,...
10989,CC(=NOCC1=CC=CC=C1C(=NOC)C(=O)OC)C2=CC(=CC=C2)...,CC(=NOCC1=CC=CC=C1C(=NOC)C(=O)OC)C2=CC(=CC=C2)...,38
10990,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)...,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)...,34
10991,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,1640
10992,COP(=O)(OC)OC=C(Cl)Cl;28,COP(=O)(OC)OC=C(Cl)Cl,28


In [40]:
# calculate the Morgan fingerprints for the test set
test_Morgan_fpts = morgan_fpts(test_data.Chemical_Id)
print(test_Morgan_fpts.shape)

# create a dataframe from the test fingerprints
test_fp = pd.DataFrame(test_Morgan_fpts,columns=['Col_{}'.format(i) for i in range(test_Morgan_fpts.shape[1])])
test_fp

(10994, 512)


,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Col_176,Col_177,Col_178,Col_179,Col_180,Col_181,Col_182,Col_183,Col_184,Col_185,Col_186,Col_187,Col_188,Col_189,Col_190,Col_191,Col_192,Col_193,Col_194,Col_195,Col_196,Col_197,Col_198,Col_199,Col_200,Col_201,Col_202,Col_203,Col_204,Col_205,Col_206,Col_207,Col_208,Col_209,Col_210,Col_211,Col_212,Col_213,Col_214,Col_215,Col_216,Col_217,Col_218,Col_219,Col_220,Col_221,Col_222,Col_223,Col_224,Col_225,Col_226,Col_227,Col_228,Col_229,Col_230,Col_231,Col_232,Col_233,Col_234,Col_235,Col_236,Col_237,Col_238,Col_239,Col_240,Col_241,Col_242,Col_243,Col_244,Col_245,Col_246,Col_247,Col_248,Col_249,Col_250,Col_251,Col_252,Col_253,Col_254,Col_255,Col_256,Col_257,Col_258,Col_259,Col_260,Col_261,Col_262,Col_263,Col_264,Col_265,Col_266,Col_267,Col_268,Col_269,Col_270,Col_271,Col_272,Col_273,Col_274,Col_275,Col_276,Col_277,Col_278,Col_279,Col_280,Col_281,Col_282,Col_283,Col_284,Col_285,Col_286,Col_287,Col_288,Col_289,Col_290,Col_291,Col_292,Col_293,Col_294,Col_295,Col_296,Col_297,Col_298,Col_299,Col_300,Col_301,Col_302,Col_303,Col_304,Col_305,Col_306,Col_307,Col_308,Col_309,Col_310,Col_311,Col_312,Col_313,Col_314,Col_315,Col_316,Col_317,Col_318,Col_319,Col_320,Col_321,Col_322,Col_323,Col_324,Col_325,Col_326,Col_327,Col_328,Col_329,Col_330,Col_331,Col_332,Col_333,Col_334,Col_335,Col_336,Col_337,Col_338,Col_339,Col_340,Col_341,Col_342,Col_343,Col_344,Col_345,Col_346,Col_347,Col_348,Col_349,Col_350,Col_351,Col_352,Col_353,Col_354,Col_355,Col_356,Col_357,Col_358,Col_359,Col_360,Col_361,Col_362,Col_363,Col_364,Col_365,Col_366,Col_367,Col_368,Col_369,Col_370,Col_371,Col_372,Col_373,Col_374,Col_375,Col_376,Col_377,Col_378,Col_379,Col_380,Col_381,Col_382,Col_383,Col_384,Col_385,Col_386,Col_387,Col_388,Col_389,Col_390,Col_391,Col_392,Col_393,Col_394,Col_395,Col_396,Col_397,Col_398,Col_399,Col_400,Col_401,Col_402,Col_403,Col_404,Col_405,Col_406,Col_407,Col_408,Col_409,Col_410,Col_411,Col_412,Col_413,Col_414,Col_415,Col_416,Col_417,Col_418,Col_419,Col_420,Col_421,Col_422,Col_423,Col_424,Col_425,Col_426,Col_427,Col_428,Col_429,Col_430,Col_431,Col_432,Col_433,Col_434,Col_435,Col_436,Col_437,Col_438,Col_439,Col_440,Col_441,Col_442,Col_443,Col_444,Col_445,Col_446,Col_447,Col_448,Col_449,Col_450,Col_451,Col_452,Col_453,Col_454,Col_455,Col_456,Col_457,Col_458,Col_459,Col_460,Col_461,Col_462,Col_463,Col_464,Col_465,Col_466,Col_467,Col_468,Col_469,Col_470,Col_471,Col_472,Col_473,Col_474,Col_475,Col_476,Col_477,Col_478,Col_479,Col_480,Col_481,Col_482,Col_483,Col_484,Col_485,Col_486,Col_487,Col_488,Col_489,Col_490,Col_491,Col_492,Col_493,Col_494,Col_495,Col_496,Col_497,Col_498,Col_499,Col_500,Col_501,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,0,0,0,0,0,0

In [41]:
# dropping the Id and Chemical_Id columns for the train and test data
train_data = train_data.drop(columns=['Id','Chemical_Id'], axis=1)
test_data =  test_data.drop(columns=['Id','Chemical_Id'], axis=1)

In [47]:
# merging the datases
train_fp_desc=train_desc.join(train_fp,how="outer")
train_fp_desc.head()

train_final=train_fp_desc.join(train_data,how="outer")
train_final.sample(10)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Col_176,Col_177,Col_178,Col_179,Col_180,Col_181,Col_182,Col_183,Col_184,Col_185,Col_186,Col_187,Col_188,Col_189,Col_190,Col_191,Col_192,Col_193,Col_194,Col_195,Col_196,Col_197,Col_198,Col_199,Col_200,Col_201,Col_202,Col_203,Col_204,

In [48]:
test_fp_desc=test_desc.join(test_fp,how="outer")
test_fp_desc.head()

test_final=test_fp_desc.join(test_data,how="outer")
test_final.sample(10)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14,Col_15,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25,Col_26,Col_27,Col_28,Col_29,Col_30,Col_31,Col_32,Col_33,Col_34,Col_35,Col_36,Col_37,Col_38,Col_39,Col_40,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Col_51,Col_52,Col_53,Col_54,Col_55,Col_56,Col_57,Col_58,Col_59,Col_60,Col_61,Col_62,Col_63,Col_64,Col_65,Col_66,Col_67,Col_68,Col_69,Col_70,Col_71,Col_72,Col_73,Col_74,Col_75,Col_76,Col_77,Col_78,Col_79,Col_80,Col_81,Col_82,Col_83,Col_84,Col_85,Col_86,Col_87,Col_88,Col_89,Col_90,Col_91,Col_92,Col_93,Col_94,Col_95,Col_96,Col_97,Col_98,Col_99,Col_100,Col_101,Col_102,Col_103,Col_104,Col_105,Col_106,Col_107,Col_108,Col_109,Col_110,Col_111,Col_112,Col_113,Col_114,Col_115,Col_116,Col_117,Col_118,Col_119,Col_120,Col_121,Col_122,Col_123,Col_124,Col_125,Col_126,Col_127,Col_128,Col_129,Col_130,Col_131,Col_132,Col_133,Col_134,Col_135,Col_136,Col_137,Col_138,Col_139,Col_140,Col_141,Col_142,Col_143,Col_144,Col_145,Col_146,Col_147,Col_148,Col_149,Col_150,Col_151,Col_152,Col_153,Col_154,Col_155,Col_156,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166,Col_167,Col_168,Col_169,Col_170,Col_171,Col_172,Col_173,Col_174,Col_175,Col_176,Col_177,Col_178,Col_179,Col_180,Col_181,Col_182,Col_183,Col_184,Col_185,Col_186,Col_187,Col_188,Col_189,Col_190,Col_191,Col_192,Col_193,Col_194,Col_195,Col_196,Col_197,Col_198,Col_199,Col_200,Col_201,Col_202,Col_203,Col_204,

In [53]:
# save to a csv file
train_final.to_csv('train_final.csv', index=False)
test_final.to_csv('test_final.csv', index=False)

## Modelling

In [14]:
import pandas as pd
import numpy as np

In [4]:
# load the train and test data from saved checkpoint
train_final = pd.read_csv('train_final.csv')
test_final = pd.read_csv('test_final.csv')

In [5]:

X_train = train_final.drop(columns=['Expected'], axis=1)
y_train = train_final['Expected']

X_test = test_final

### Cleaning the data

In [6]:
print(X_train.isna().sum())

MaxEStateIndex       6
MinEStateIndex       6
MaxAbsEStateIndex    6
MinAbsEStateIndex    6
qed                  6
                    ..
Col_508              6
Col_509              6
Col_510              6
Col_511              6
Assay_Id             6
Length: 721, dtype: int64


In [7]:
print(y_train.isna().sum())

6


In [8]:
# filling the missing values with the mean
X_train.fillna(X_train.mean(), inplace=True)
y_train.fillna(y_train.mean(), inplace=True)

### Scaling the Data

In [10]:
# scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
y_train = scaler.fit_transform(y_train.values.reshape(-1,1))

### 1. Base Model

In [ ]:
from flaml import AutoML
clf = AutoML()
clf.fit(X_train, y_train, task="classification", time_budget=10000, estimator_list=['lgbm'])

In [ ]:
# Evaluating the model
from sklearn.metrics import *

y_pred = clf.predict(X_test)

# classification report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### 2. XGBoost Classifier

In [16]:
print(np.unique(y_train))

[-2.46430992  0.          0.40582542]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

# Print the unique values of the encoded y
print(np.unique(y_encoded))

In [19]:
# XGBoost model
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [24]:
from sklearn.model_selection import cross_val_score

# Evaluate the performance of the classifier using cross-validation
scores = cross_val_score(xgb, X_train, y_train, cv=5, scoring="accuracy")

print(scores)

# Print the average accuracy across all folds
print("\nAverage accuracy:", np.mean(scores))

[0.86549048 0.8678119  0.86343437 0.86415495 0.86103741]

Average accuracy: 0.8643858220117892


### 2. Model with PCA

In [ ]:
from sklearn.decomposition import PCA

# Create a PCA object with 2 components
pca = PCA(n_components=6)

# Fit the PCA model to the training data
pca.fit(X_train)

# Transform the training and test data using the PCA model
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [26]:
model_pca_xgb = xgb.fit(X_train_pca, y_train)

In [27]:
from sklearn.model_selection import cross_val_score

# Evaluate the performance of the classifier using cross-validation
scores = cross_val_score(model_pca_xgb, X_train_pca, y_train, cv=5, scoring="accuracy")

print(scores)

# Print the average accuracy across all folds
print("\nAverage accuracy:", np.mean(scores))

[0.85786297 0.85859256 0.8579293  0.85805253 0.85825153]

Average accuracy: 0.8581377767998865


### 3. Model with Neural Network

In [43]:
from keras.models import Sequential
from keras.layers import Dense

network.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
network.add(Dense(256, activation='relu'))
network.add(Dense(1, activation='sigmoid')) 

In [44]:
network.compile(optimizer='rmsprop', # try adam too
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [45]:
# Train the model
network_model = network.fit(X_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
589/589 [==============================] - 14s 20ms/step - loss: -1927.2816 - accuracy: 7.9594e-05
Epoch 2/20
589/589 [==============================] - 13s 21ms/step - loss: -22879.9121 - accuracy: 7.9594e-05
Epoch 3/20
589/589 [==============================] - 12s 19ms/step - loss: -90755.8906 - accuracy: 7.9594e-05
Epoch 4/20
589/589 [==============================] - 10s 17ms/step - loss: -232035.1406 - accuracy: 7.9594e-05
Epoch 5/20
589/589 [==============================] - 15s 25ms/step - loss: -476003.6250 - accuracy: 7.9594e-05
Epoch 6/20
589/589 [==============================] - 14s 24ms/step - loss: -849616.4375 - accuracy: 7.9594e-05
Epoch 7/20
589/589 [==============================] - 9s 16ms/step - loss: -1378747.6250 - accuracy: 7.9594e-05
Epoch 8/20
589/589 [==============================] - 10s 17ms/step - loss: -2091010.5000 - accuracy: 7.9594e-05
Epoch 9/20
589/589 [==============================] - 13s 22ms/step - loss: -3016760.5000 - accuracy: 7.959

In [49]:
# Evaluate the model on the test set
test_loss, test_accuracy = network.evaluate(X_train, y_train)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

2356/2356 [==============================] - 10s 4ms/step - loss: -37755284.0000 - accuracy: 7.9594e-05
Test loss: -37755284.0
Test accuracy: 7.959354115882888e-05


In [50]:
network.predict(X_test)

344/344 [==============================] - 1s 4ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)